In [1]:
%pip install pycaret

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pycaret.regression as reg
import xgboost as xgb

In [3]:
df = pd.read_csv('MiningProcess_Flotation_Plant_Database.csv', decimal=',')
target = '% Silica Concentrate'
print(df.shape)
df.drop(['date'], inplace=True, axis=1)

df.head()


(737453, 24)


,% Iron Feed,% Silica Feed,Starch Flow,Amina Flow,Ore Pulp Flow,Ore Pulp pH,Ore Pulp Density,Flotation Column 01 Air Flow,Flotation Column 02 Air Flow,Flotation Column 03 Air Flow,...,Flotation Column 07 Air Flow,Flotation Column 01 Level,Flotation Column 02 Level,Flotation Column 03 Level,Flotation Column 04 Level,Flotation Column 05 Level,Flotation Column 06 Level,Flotation Column 07 Level,% Iron Concentrate,% Silica Concentrate
0,55.2,16.98,3019.53,557.434,395.713,10.0664,1.74,249.214,253.235,250.576,...,250.884,457.396,432.962,424.954,443.558,502.255,446.370,523.344,66.91,1.31
1,55.2,16.98,3024.41,563.965,397.383,10.0672,1.74,249.719,250.532,250.862,...,248.994,451.891,429.560,432.939,448.086,496.363,445.922,498.075,66.91,1.31
2,55.2,16.98,3043.46,568.054,399.668,10.0680,1.74,249.741,247.874,250.313,...,248.071,451.240,468.927,434.610,449.688,484.411,447.826,458.567,66.91,1.31
3,55.2,16.98,3047.36,568.665,397.939,10.0689,1.74,249.917,254.487,250.049,...,251.147,452.441,458.165,442.865,446.210,471.411,437.690,427.669,66.91,1.31
4,55.2,16.98,3033.69,558.167,400.254,10.0697,1.74,250.203,252.136,249.895,...,248.928,452.441,452.900,450.523,453.670,462.598,443.682,425.679,66.91,1.31


In [4]:
import pandas as pd

def eliminar_outliers_iqr(df):
    # Crear una copia del DataFrame para no modificar el original
    df_sin_outliers = df.copy()

    # Iterar sobre cada columna del DataFrame
    for col in df_sin_outliers.columns:
        if df_sin_outliers[col].dtype in ['float64', 'int64']:  # Solo aplicar a columnas numéricas
            # Calcular Q1 (primer cuartil) y Q3 (tercer cuartil)
            Q1 = df_sin_outliers[col].quantile(0.25)
            Q3 = df_sin_outliers[col].quantile(0.75)
            IQR = Q3 - Q1  # Rango Intercuartílico

            # Definir los límites inferior y superior para detectar los outliers
            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR

            # Filtrar los valores que no sean outliers
            df_sin_outliers = df_sin_outliers[(df_sin_outliers[col] >= limite_inferior) & (df_sin_outliers[col] <= limite_superior)]

    return df_sin_outliers

In [5]:
df_parsed = df[['% Silica Feed', 'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow', 'Flotation Column 03 Air Flow',
                'Flotation Column 04 Level', 'Flotation Column 05 Level', 'Flotation Column 06 Level', 'Flotation Column 07 Level',
                # '% Iron Concentrate',
                 target]]

df_parsed = eliminar_outliers_iqr(df_parsed).sample(10000)

print('Dimensiones de la data original',df.shape)
print('Dimensiones de la data limpia',df_parsed.shape)
df_parsed.head()

Dimensiones de la data original (737453, 23)
Dimensiones de la data limpia (10000, 10)


,% Silica Feed,% Iron Feed,Flotation Column 01 Air Flow,Flotation Column 02 Air Flow,Flotation Column 03 Air Flow,Flotation Column 04 Level,Flotation Column 05 Level,Flotation Column 06 Level,Flotation Column 07 Level,% Silica Concentrate
110270,18.69,52.17,220.386,222.363,222.363,401.953,402.070,192.660,328.848,2.25
594017,25.31,48.81,297.427,298.031,301.533,460.814,432.195,503.771,502.067,1.58
711312,15.13,55.99,298.828,292.627,301.685,387.258,386.453,450.507,411.441,4.77
72312,15.26,55.30,250.267,249.706,249.991,449.473,452.745,400.911,429.370,1.04
71625,15.26,55.30,250.287,249.114,250.738,464.877,458.660,456.352,430.664,1.01


In [6]:
setup = reg.setup(df_parsed, target=target, session_id=123)

,Description,Value
0,Session id,123
1,Target,% Silica Concentrate
2,Target type,Regression
3,Original data shape,"(10000, 10)"
4,Transformed data shape,"(10000, 10)"
5,Transformed train set shape,"(7000, 10)"
6,Transformed test set shape,"(3000, 10)"
7,Numeric features,9
8,Preprocess,True
9,Imputation type,simple


In [7]:
model = setup.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.6223,0.6983,0.8354,0.4296,0.2389,0.3086,0.0990
rf,Random Forest Regressor,0.6493,0.7322,0.8553,0.4018,0.2444,0.3219,0.2620
lightgbm,Light Gradient Boosting Machine,0.6621,0.7430,0.8618,0.3932,0.2450,0.3232,0.1910
xgboost,Extreme Gradient Boosting,0.6566,0.7545,0.8683,0.3837,0.2488,0.3216,0.0930
gbr,Gradient Boosting Regressor,0.7293,0.8566,0.9253,0.3004,0.2632,0.3595,0.1750
br,Bayesian Ridge,0.8348,1.0923,1.0450,0.1077,0.2962,0.4137,0.0040
lr,Linear Regression,0.8346,1.0923,1.0450,0.1076,0.2961,0.4135,0.3150
ridge,Ridge Regression,0.8346,1.0923,1.0450,0.1076,0.2961,0.4135,0.2210
lar,Least Angle Regression,0.8348,1.0927,1.0452,0.1073,0.2962,0.4134,0.0080
en,Elastic Net,0.8386,1.0987,1.0481,0.1024,0.2968,0.4156,0.1370


In [8]:
reg.tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.7824,0.9559,0.9777,0.2245,0.2789,0.3900
1,0.7756,0.9461,0.9727,0.2015,0.2823,0.4030
2,0.7980,1.0135,1.0067,0.1788,0.2853,0.3954
3,0.7620,0.9257,0.9621,0.2597,0.2737,0.3797
4,0.8013,1.0223,1.0111,0.1856,0.2800,0.3753
5,0.7756,0.9507,0.9750,0.2103,0.2804,0.3948
6,0.7623,0.9233,0.9609,0.2421,0.2729,0.3782
7,0.7829,0.9807,0.9903,0.2021,0.2770,0.3727
8,0.7877,0.9756,0.9877,0.1693,0.2863,0.4108


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


ExtraTreesRegressor(n_jobs=-1, random_state=123)

In [9]:
reg.evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [10]:
xg = reg.create_model('xgboost')
reg.tune_model(xg)
reg.evaluate_model(xg)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.6424,0.7041,0.8391,0.4288,0.2409,0.3106
1,0.6533,0.7526,0.8675,0.3648,0.2525,0.3344
2,0.6768,0.7869,0.8871,0.3624,0.2550,0.3338
3,0.6384,0.7133,0.8446,0.4295,0.2421,0.3083
4,0.6720,0.8279,0.9099,0.3405,0.2547,0.3101
5,0.6616,0.7674,0.8760,0.3625,0.2540,0.3295
6,0.6534,0.7223,0.8499,0.4071,0.2463,0.3260
7,0.6479,0.7376,0.8588,0.3999,0.2413,0.3024
8,0.6532,0.7383,0.8592,0.3713,0.2521,0.3415


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.6429,0.7148,0.8455,0.4201,0.2438,0.3148
1,0.6690,0.7823,0.8845,0.3397,0.2582,0.3414
2,0.6868,0.7959,0.8922,0.3551,0.2563,0.3384
3,0.6559,0.7448,0.8630,0.4044,0.2484,0.3188
4,0.6932,0.8594,0.9270,0.3154,0.2606,0.3203
5,0.6884,0.8002,0.8945,0.3353,0.2605,0.3436
6,0.6399,0.7109,0.8432,0.4164,0.2424,0.3140
7,0.6693,0.7780,0.8821,0.3670,0.2481,0.3119
8,0.6732,0.7645,0.8744,0.3490,0.2560,0.3460


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…